# Day 12 - Simulation with vectors

Part 1 is straight up simulation work; calculate in steps, following the same operations each step.

This is a good job for numpy again, where we can represent the vectors for position and velocity for each moon in a 4 x 2 x 3 matrix.

Updating the velocities is a question of summing the [signs](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sign.html) of the subtracted positions per moon; e.g. for the starting point of the first example, the $x$ values for the moons are `-1`, `2`, `4` and `3`. So for the first moon, the delta-v for the gravity effect of the other moons is `sign(-1 - 2) + sign(-1 - 4) + sign(-1 - 3)`, which is `-1 + -1 + -1` is `-3`.

This is of course easily vectorised, the index `[:, np.newaxis, 0]` gives us the positions of each moon in a 4 x 1 x 3 matrix so we can subtract `[:, 0]`, which is a 4 x 3 matrix of the moon positions. `np.sign()` of that subtraction is then a 4 x 4 x 3 matrix of -1, 0 and 1 values, a 4 x 3 matrix per moon (the 4th vector there is the `[0, 0, 0]`  result of subtraction from itself), so we can sum these across the first axis to the expected 4 x 3 delta-vs.

In [1]:
from __future__ import annotations
import re
from io import StringIO
from typing import Pattern, Sequence

import numpy as np


_vec = re.compile(r"<x=\s?(?P<x>-?\d+),\s*y=\s?(?P<y>-?\d+),\s*z=\s?(?P<z>-?\d+)>")


def load_moons(data: str, _vec: Pattern[str] = _vec) -> np.array:
    positions = np.fromregex(StringIO(data), _vec, dtype=np.int)
    return np.stack((positions, np.zeros(positions.shape, positions.dtype)), axis=1)


def step(moons: np.array) -> None:
    # calculate the delta-v based on positions, adjust velocities
    # the delta is the sum of the signs of the difference between positions
    # of each moon relative to the other moons.
    moons[:, 1] += np.sign(moons[:, np.newaxis, 0] - moons[:, 0]).sum(axis=0)
    # adjust positions
    moons[:, 0] += moons[:, 1]
        

def total_energy(moons: np.array) -> int:
    # per moon, the product of the absolute sums of the position and velocity vectors
    # so abs(moons) summed over axis 2 (the 2 vector types), then multiplied
    # over axis 1 (vector energy per moon), then summed.
    return np.abs(moons).sum(axis=2).prod(axis=1).sum()


test1_moons = load_moons("""\
<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
""")
for _ in range(10):
    step(test1_moons)
assert total_energy(test1_moons) == 179


test2_moons = load_moons("""\
<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
""")
for _ in range(100):
    step(test2_moons)
assert total_energy(test2_moons) == 1940

In [2]:
import aocd
moons = load_moons(aocd.get_data(day=12, year=2019))

In [3]:
energy_moons = moons.copy()
for _ in range(1000):
    step(energy_moons)
print("Part 1:", total_energy(energy_moons))

Part 1: 10028


## Part 2 - finding the cycles

Part 1 was simple enough; 1000 steps of simulation is trivial enough. While numpy vectorised operations are very efficient, I somehow think we won't get to solve part 2 using brute force.

But... the $x$, $y$ and $z$ components of our system are entirely independent. They must each have their own periodicity, and so the system *as a whole* will repeat at the next common multiple for the periods of each dimension.

I've inlined the `step()` calculations as we are still going to have to run our simulation a few hundred-thousand  times. I'm running the simulation on all dimensions as a whole but look for the cycles across each dimension. The cycles lie too close together for removing dimensions from the calculations once a cycle has been found for it to help improve speeds. Calculating part 2 takes a little under 7 seconds with this approach, complicating matters by using `np.delete()` only saves about 400ms.

In [4]:
from itertools import count

def find_cycle(moons: nd.array) -> int:
    # caches, to avoid repeated lookups of globals
    sign, newaxis, array_equal = np.sign, np.newaxis, np.array_equal

    sim = moons.copy()
    dims = list(reversed(range(moons.shape[-1])))
    period = 1

    for i in count(1):
        # inlined from step() for better speed
        sim[:, 1] += sign(sim[:, newaxis, 0] - sim[:, 0]).sum(axis=0)
        sim[:, 0] += sim[:, 1]

        for d in dims:
            if array_equal(sim[..., d], moons[..., d]):
                period = np.lcm(period, i)
                dims.remove(d)
                if not dims:
                    return period


tests = {
    "<x=-1, y=0, z=2>\n<x=2, y=-10, z=-7>\n<x=4, y=-8, z=8>\n<x=3, y=5, z=-1>": 2772,
    "<x=-8, y=-10, z=0>\n<x=5, y=5, z=10>\n<x=2, y=-7, z=3>\n<x=9, y=-8, z=-3>": 4686774924,
}

for testspec, expected in tests.items():
    assert find_cycle(load_moons(testspec)) == expected

In [5]:
print("Part 2:", find_cycle(moons))

Part 2: 314610635824376
